<a href="https://colab.research.google.com/github/ali-rabiee/Portfolio-Formation/blob/main/Portfolio_Formation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements & Libraries

In [1]:
!pip3 install keras-rl
!pip install keras
!pip install keras-rl
!pip install gym
!pip install rl
!pip install keras-rl2
!pip install Callbacks 
!pip install callbacks
!pip install rl.callbacks
!pip install tf-nightly
!pip install yfinance
!pip install minisom

  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Attempting uninstall: tf-estimator-nightly
    Found existing installation: tf-estimator-nightly 2.10.0.dev2022051408
    Uninstalling tf-estimator-nightly-2.10.0.dev2022051408:
      Successfully uninstalled tf-estimator-nightly-2.10.0.dev2022051408
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.10.0.dev20220514 requires tf-estimator-nightly~=2.10.0.dev, but you have tf-estimator-nightly 2.8.0.dev2021122109 which is incompatible.
ERROR: Could not find a version that satisfies the requirement rl.callbacks (from versions: none)
ERROR: No matching distribution found for rl.callbacks
  Using cached tf_estimator_nightly-2.10.0.dev2022051408-py2.py3-none-any.whl (438 kB)
  Attempting uninstall: tf-estimator-nightly
    Found existing installation: tf-estimator-night

# Google Drive Connection

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive/', force_remount=True)
os.chdir('drive/My Drive/Colab Notebooks/DQN')

Mounted at /content/drive/


# Get Data

In [2]:
# Name of 60 Stocks  
stocks = ['CDEV', 'GILD', 'JBLU', 'HBAN','SLB',
          'CS', 'PLUG', 'USB', 'ABBV','CTRA', 'SIRI',
          'SCHW', 'M', 'FCX', 'MO','UAL', 'ET','KGC',
          'NCLH', 'NKE','ERIC', 'ZNGA','PTON',
          'CLF', 'PYPL', 'MRO','CMCSA', 'INFY',
          'SNAP','TSM', 'CSCO', 'FB', 'UBER', 'OXY',
          'JPM', 'GOLD', 'TSLA', 'BABA', 'NLSN','TLRY',
          'AMC', 'VALE', 'DAL', 'TELL','ITUB', 'PCG','AAL',
          'NVDA', 'WFC', 'AMD', 'AAPL', 'TWTR', 'MRNA',
          'JNJ', 'GOOGL', 'COST', 'WMT']

print(f"Number of Stocks: {len(stocks)}")

Number of Stocks: 57


In [ ]:
from get_stock import get_stock

# Getting Data

start = "2020-05-10"
end = "2022-04-29"

for ticker in stocks:
    # Download datasets
    print(f'### Downloading {ticker}:')
    Hour = get_stock(ticker, start, end, "60m")
    Day = get_stock(ticker, start, end, "1d")
    Week = get_stock(ticker, start, end, "1wk")
    
    # Reset indexes
    Hour.reset_index(drop=True, inplace=True)
    Day.reset_index(drop=True, inplace=True)
    Week.reset_index(drop=True, inplace=True)

    # Save the datasets
    PATH = "datasets/"
    Hour.to_csv(f"{PATH}{ticker}hour.csv")
    Day.to_csv(f"{PATH}{ticker}day.csv")
    Week.to_csv(f"{PATH}{ticker}week.csv")

# Train the DQN Model

The code needs three positional parameters to be correctly executed: \\
python main.py [numberOfActions, isOnlyShort, ensembleFolder]


* To run the FULL agent you need to run: python main.py 3 0 ensembleFolder
* To run the ONLY LONG agent you need to run: python main.py 2 0 ensembleFolder
* To run the ONLY SHORT agent you need to run: python main.py 2 1 ensembleFolder \\
where the paramenter ensembleFolder is used to set the name of the folder in which you'll get your results.

In [ ]:
"""This is the class call for the Agent which will perform the experiment"""

#This is the class call for the Agent which will perform the experiment
from deepQTrading import DeepQTrading

#Date library to manipulate time in the source code
import datetime

#Keras library to define the NN to be used
from keras.models import Sequential

#Layers used in the NN considered
from keras.layers import Dense, Activation, Flatten

#Activation Layers used in the source code
from keras.layers.advanced_activations import LeakyReLU, PReLU, ReLU

#Optimizer used in the NN
from keras.optimizers import Adam

#Libraries used for the Agent considered
from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy


#Library used for showing the exception in the case of error 
import sys

import pandas as pd
import numpy as np
from evaluation import perc_ensemble, full_ensemble, ensemble

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

'''
There are three actions possible in the stock market
Hold(id 0): do nothing.
Long(id 1): It predicts that the stock market value will raise at the end of the day. 
So, the action performed in this case is buying at the beginning of the day and sell it at the end of the day (aka long).
Short(id 2): It predicts that the stock market value will decrease at the end of the day.
So, the action that must be done is selling at the beginning of the day and buy it at the end of the day (aka short). 
//////////////////////////////////////////////////////////////
The Model is a simple NN considered. It is composed of:
One flatten layer to get 68 dimensional vectors as input
One dense layer with 35 neurons and LeakyRelu activation
One final Dense Layer with the 3 actions considered
the input is 20 observation days from the past, 8 observations from the past week and 
40 observations from the past hours
//////////////////////////////////////////////////////////////
Define the DeepQTrading class with the following parameters:
explorations: 0.2 operations are random, and 100 epochs.
in this case, epochs parameter is used because the Agent acts on daily basis, so its better to repeat the experiments several
times so, its defined that each epoch will work on the data from training, validation and testing.
trainSize: the size of the train data gotten from the dataset, we are setting 5 stock market years, or 1800 days
validationSize: the size of the validation data gotten from dataset, we are setting 6 stock market months, or 180 days
testSize: the size of the testing data gotten from dataset, we are setting 6 stock market months, or 180 days
outputFile: where the results will be written
begin: where the walks will start from. We are defining January 1st of 2010
end: where the walks will finish. We are defining February 22nd of 2019
nOutput:number of walks
'''
# Set Parameters
nb_actions = 3
isOnlyShort = 0
ensembleFolderName = "ensembleFolder"


# Define the DQN model 
model = Sequential()
model.add(Flatten(input_shape=(1, 1, 68)))
model.add(Dense(50, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dense(50, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dense(nb_actions))
model.add(Activation('linear'))


stocks = ['MRO','CMCSA', 'INFY',
          'SNAP', 'TSM', 'CSCO', 'FB', 'UBER', 'OXY',
          'JPM', 'GOLD', 'TSLA', 'BABA', 'NLSN','TLRY',
          'AMC', 'VALE', 'DAL', 'TELL','ITUB', 'PCG','AAL',
          'NVDA', 'WFC', 'AMD', 'AAPL', 'TWTR', 'MRNA',
          'JNJ', 'GOOGL', 'COST', 'WMT']

for ticker in stocks:

    print(f"\n#### Train {ticker} ####\n")
    # Define the DeepQTrading class
    dqt = DeepQTrading(
        ticker=ticker,
        model=model,
        explorations=[(0.2, 50)],
        trainSize=datetime.timedelta(days=30*19),
        validationSize=datetime.timedelta(days=30*1),
        testSize=datetime.timedelta(days=30*1),
        outputFile="Output/csv/walks/walks",
        begin=datetime.datetime(2020, 6, 30, 0, 0, 0, 0),
        end=datetime.datetime(2022, 4, 1, 0, 0, 0, 0),
        nbActions=nb_actions,
        isOnlyShort=isOnlyShort,
        ensembleFolderName=ensembleFolderName
        )
    numWalks = dqt.run()
    dqt.end()

    # Preparing and saving the results
    numDel = 0
    for j in range(0, numWalks):

        df_test = pd.read_csv("./Output/ensemble/ensembleFolder/walk"+str(j)+f"ensemble_test_{ticker}.csv", index_col='Date')
        df_valid = pd.read_csv("./Output/ensemble/ensembleFolder/walk"+str(j)+f"ensemble_valid_{ticker}.csv", index_col='Date')

        for deleted in range(1, numDel):
            del df_test['iteration'+str(deleted)]
            del df_valid['iteration'+str(deleted)]

        if j == 0:
            fulldf_test = perc_ensemble(df_test)
            fulldf_valid = perc_ensemble(df_valid)

        else:
            fulldf_test = fulldf_test.append(perc_ensemble(df_test))
            fulldf_valid = fulldf_valid.append(perc_ensemble(df_valid))


    metrics_test = ensemble(numWalks, ticker, data_type='test', numDel=10)
    metrics_valid = ensemble(numWalks, ticker, data_type='valid', numDel=10)

    # method = "short" if isOnlyShort == 1 else "long"
    # method = '3act' if nb_actions == 3 else method 

    # Save Predictions
    PATH = "./Output/results/DQN predictions/"
    fulldf_test.to_csv(f"{PATH}{ticker}_preds_test.csv")
    fulldf_valid.to_csv(f"{PATH}{ticker}_preds_valid.csv")

    # Save metrics
    PATH = "./Output/results/DQN metrics/"
    metrics_test.to_csv(f"{PATH}{ticker}_metrics_test.csv")
    metrics_valid.to_csv(f"{PATH}{ticker}_metrics_valid.csv")

# Portfolio Selection

## Extract Profitable Stocks

In [4]:
stocks = ['CDEV', 'GILD', 'JBLU', 'HBAN','SLB',
          'CS', 'PLUG', 'USB', 'ABBV','CTRA', 'SIRI',
          'SCHW', 'M', 'FCX', 'MO','UAL', 'ET','KGC',
          'NCLH', 'NKE','ERIC', 'ZNGA','PTON',
          'CLF', 'PYPL', 'MRO','CMCSA', 'INFY',
          'SNAP','TSM', 'CSCO', 'FB', 'UBER', 'OXY',
          'JPM', 'GOLD', 'TSLA', 'BABA', 'NLSN','TLRY',
          'AMC', 'VALE', 'DAL', 'TELL','ITUB', 'PCG','AAL',
          'NVDA', 'WFC', 'AMD', 'AAPL', 'TWTR', 'MRNA',
          'JNJ', 'GOOGL', 'COST', 'WMT']

In [ ]:
# Ensemble the actions for only-long and only-short agents

data_type = 'test'

for ticker in stocks:
    preds_long = pd.read_csv(f"./Output/results/DQN predictions/{ticker}_long_preds_{data_type}.csv")
    preds_short = pd.read_csv(f"./Output/results/DQN predictions/{ticker}_short_preds_{data_type}.csv")
    preds_ensemble = []

    for i in range(len(preds_long)):
        if preds_long["ensemble"][i] == 1 and preds_short["ensemble"][i] == 0:
            preds_ensemble.append(1)
        elif preds_long["ensemble"][i] == 0 and preds_short["ensemble"][i] == 2:
            preds_ensemble.append(2)
        else:
            preds_ensemble.append(0) 

    # Read predictions file
    preds = pd.read_csv(f"./Output/results/DQN predictions/{ticker}_preds_{data_type}.csv")
    preds['ensemble preds'] = preds_ensemble
    # Save 
    PATH = "./Output/results/DQN prediction/"
    preds.to_csv(f"{PATH}{ticker}_preds_{data_type}.csv")

In [ ]:
#####################
#     Comparison    #
#####################

In [5]:
# Extract Profitable stocks in each day

import pandas as pd

method = "short"
data_type = "test"

# Date of the preds
preds = pd.read_csv(f"./Output/results/DQN predictions/{stocks[0]}_preds_{data_type}.csv")
Date = preds_3act["Date"].to_list()
portfolio_ensemble = {}
portfolio_3act = {}
 
for date in Date:
    
    profitable_stocks1 = []
    profitable_stocks2 = []

    for ticker in stocks:
        # Read Predictions
        preds_3act = pd.read_csv(f"./Output/results/DQN predictions/{ticker}_preds_{data_type}.csv")
        preds_long = pd.read_csv(f"./Output/results/DQN predictions/{ticker}_long_preds_{data_type}.csv")
        preds_short = pd.read_csv(f"./Output/results/DQN predictions/{ticker}_short_preds_{data_type}.csv")
        # preds = pd.read_csv(f"./Output/results/predictions/{ticker}_{method}_preds_{data_type}.csv")

        # Get Predictions
        pred_3act = int(preds_3act[preds_3act["Date"] == date]["ensemble"])
        pred_long = int(preds_long[preds_3act["Date"] == date]["ensemble"])
        pred_short = int(preds_short[preds_3act["Date"] == date]["ensemble"])

        if pred_long == 1 and pred_short == 0:
            profitable_stocks1.append(ticker)

        if pred_3act == 1:
            profitable_stocks2.append(ticker)

    portfolio_ensemble[date] = profitable_stocks1
    portfolio_3act[date] = profitable_stocks2

In [ ]:
# Rank the Rewards base on validation set

data_type = 'valid'
columns = ["Iteration", "Reward_Long%", "Reward_Short%", "Reward%", "Wins%", "Losses%", "Coverage%", "Accuracy", "Dataset"]
df_metrics = pd.DataFrame(columns=columns)
for ticker in stocks:
    metrics = pd.read_csv(f"./Output/results/DQN metrics/{ticker}_metrics_{data_type}.csv")
    metrics = metrics[[metrics['Iteration']=='sum']] 
    metrics['Dataset'] = ticker
    df_metrics.append(metrics)

## Correlation

In [ ]:
from clustering import calculate_correlation, visualize_clusters
import datetime

# Calculate the correlation of time series from the base for last num days
base = "2020-10-25"
base = datetime.datetime.strptime(base, '%Y-%m-%d')
numdays = 30
som_x, som_y, win_map, clusters = calculate_correlation(base, numdays)

In [ ]:
# Visualization
visualize_clusters(som_x, som_y, win_map)

### GRU + Attention

In [ ]:
###########################
#     Implement Model     #
###########################

### GRU

In [ ]:
###########################
#     Implement Model     #
###########################

### LSTM + Attention

In [ ]:
###########################
#     Implement Model     #
###########################

### LSTM Model

In [6]:
# Adding rewards to dataset 

stocks = ["AAPL"]

for ticker in stocks:
    df = pd.read_csv(f'./datasets/{ticker}day.csv')
    reward = []
    for i in range(len(df)):
        reward_day = (df["Close"][i] - df["Open"][i]) / df["Open"][i]
        if reward_day < 0:
            reward_day = 0
        reward.append(reward_day)
    df["reward"] = reward

df

,Unnamed: 0,Date,Time,Datetime,Open,High,Low,Close,Adj Close,Volume,reward
0,0,2020-05-11,00:00,2020-05-11,77.025002,79.262497,76.809998,78.752502,78.225410,145946400,0.022428
1,1,2020-05-12,00:00,2020-05-12,79.457497,79.922501,77.727501,77.852501,77.331436,162301200,0.000000
2,2,2020-05-13,00:00,2020-05-13,78.037498,78.987503,75.802498,76.912498,76.397720,200622400,0.000000
3,3,2020-05-14,00:00,2020-05-14,76.127502,77.447502,75.382500,77.385002,76.867065,158929200,0.016518
4,4,2020-05-15,00:00,2020-05-15,75.087502,76.974998,75.052498,76.927498,76.412621,166348400,0.024505
...,...,...,...,...,...,...,...,...,...,...,...
492,492,2022-04-22,00:00,2022-04-22,166.460007,167.869995,161.500000,161.789993,161.789993,84775200,0.000000
493,493,2022-04-25,00:00,2022-04-25,161.119995,163.169998,158.460007,162.880005,162.880005,96046400,0.010924
494,494,2022-04-26,00:00,2022-04-26,162.250000,162.339996,156.720001,156.800003,156.800003,95623200,0.000000
495,495,2022-04-27,00:00,2022-04-27,155.910004,159.789993,155.380005,156.570007,156.570007,88063200,0.004233


In [ ]:
# import matplotlib.pyplot as plt
# fig = plt.figure(1)	#identifies the figure 
# plt.figure(figsize=(20, 10))
# plt.title("Reward", fontsize='16')	#title
# plt.plot(df["reward"][:100])	#plot the points
# plt.xlabel("X",fontsize='13')	#adds a label in the x axis
# plt.ylabel("Y",fontsize='13')	#adds a label in the y axis
# plt.legend(('YvsX'),loc='best')	#creates a legend to identify the plot
# plt.grid()	#shows a grid under the plot
# plt.show()

In [ ]:
# Select usefull columns
df = df[['Open', 'Close', 'Volume']]
df

In [8]:
dataset_size = len(df)
training_size = round(dataset_size * 0.80)
train_data = df[:training_size]
test_data  = df[training_size:]

In [ ]:
training_set = df.iloc[:800, 1:2].values
test_set = df.iloc[800:, 1:2].values
look_back = 7 


# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure
X_train = []
y_train = []

for i in range(look_back, dataset_size):
    X_train.append(training_set_scaled[i - look_back:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units=50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units=1))

# Compiling the RNN
model.compile(optimizer='adam', loss='mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Portfolio Formation

## Predict Rewards

# Trading

In [ ]:
from Trading_Metrics import trade, metrics


# Load the Decisions
actions_long = pd.read_csv("resultEnsembleTest_Long.csv")
actions_short = pd.read_csv("resultEnsembleTest_Short.csv")
df = pd.read_csv("datasets/aaplDay.csv")
# Drop the useless columns
# df = df.reindex(columns = ['Date', 'Open', 'Close'])